# **Get Anchors**

In [ ]:
pip install wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=6056db1015616bd45caf880e0c65fe31bc6656691f1152054b9bce99d944d19e
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [ ]:
import wikipedia as wiki
import pandas as pd
import os

GENRE_TYPES = 'Talk-Show|News'
TITLE_TYPE = 'tvSeries'
PROFESSION_TYPES =  [' host',' presenter',' journalist',' news anchor','correspondent',' anchor']
SPLIT_TYPES = [' is a ',' is an ',' was a ',' was an ',' is the ']
# IMDB_PROFESSION_TYPES = 'actor|actress|miscellaneous|Null'
NONE = 'Not Found'
PAGE_ERROR = 'does not match any pages'
DISAMBIGUATION_ERROR = 'disambiguation'
NETWORK_TYPES = ['FOX','ABC','CBS','NBC','CNN','United Paramount Net','Warner Bros.','Pure Independent','PBS','Pax TV','Telemundo']
REGION_TYPE = 'US'
SHOW_NOT_IN_IMDB = 'Show is not present in IMDb'
NAME_BASICS = "/content/drive/My Drive/Datasets/name.basics.tsv/data.tsv"
TITLE_BASICS = "/content/drive/My Drive/Datasets/title.basics.tsv/data.tsv"
TITLE_PRINCIPALS = "/content/drive/My Drive/Datasets/title.principals.tsv.gz"
TITLE_AKAS = "/content/drive/My Drive/Datasets/title.akas.tsv/data.tsv"
TITLE_CREW = "/content/drive/My Drive/Datasets/title.crew.tsv/data.tsv"

In [ ]:
""" After getting all professions of a person we can easily confirm that
    our previous top 5 prediction is right or not & by using this we find 
    correct anchor.
"""       

def get_wiki_professions(Anchor_Name):
  if len(wiki.search(Anchor_Name)) == 0:
    return NONE

  Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
  Store_name = Read_Namebasics.primaryName.tolist()

  try:
    if Anchor_Name in Store_name:
      anchor_content = (((wiki.page(Anchor_Name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. '))
    else:
      return NONE
  except wiki.exceptions.PageError as p:
    if PAGE_ERROR in str(p):
      return NONE
  except wiki.exceptions.DisambiguationError as e:
    Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
    Store_name = Read_Namebasics.primaryName.tolist()
    if Anchor_Name in Store_name:
      # print(e.options)
      list_disambiguation_error = e.options
      for i in list_disambiguation_error:
        if i.find(DISAMBIGUATION_ERROR)!= -1:
          return NONE
        # print(i)
        anchor_content = (((wiki.page(i,auto_suggest=False)).content).replace(' and ',', ').replace('.','. '))
        # print(anchor_content)
        
        split_type = SPLIT_TYPES

        split_type_list = []
        prefix = None
        for x in split_type:
          if x in anchor_content:
            prefix = x
            split_type_index = anchor_content.find(prefix)
            #  print(index_split_type)
            pro = split_type_list.append(split_type_index)
          else:
            split_type_list.append(1000)

        low_index = split_type[split_type_list.index(min(split_type_list))]
        #  print(low_index)   
            

        if prefix is None:
          return NONE

        final_split = anchor_content.split(low_index)[1].split('. ')[0].split(', ')
        # print(final_split)
        final_professions = [anchor_content.lower() for anchor_content in final_split]
        # return final_professions
        # print(final_professions)
        required_professions = PROFESSION_TYPES

        check_individual_prof = []
        for x in required_professions:
          final_check= [(x in y) for y in final_professions]
          check_individual_prof.append(any(final_check))
          # print(x)
        
        if (any(check_individual_prof)==True):
          return final_professions
      else:
        return NONE
    
    else:
      return NONE     

  # anchor_content = (((wiki.page(name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. '))
  #  print(anchor_content)
  split_type = SPLIT_TYPES
  
  split_type_list = []
  prefix = None
  for x in split_type:
    if x in anchor_content:
      prefix = x
      split_type_index = anchor_content.find(prefix)
      #  print(index_split_type)
      pro = split_type_list.append(split_type_index)
    else:
      split_type_list.append(1000)

  low_index = split_type[split_type_list.index(min(split_type_list))]
  #  print(low_index)   
      

  if prefix is None:
    return NONE

  final_split = anchor_content.split(low_index)[1].split('. ')[0].split(', ')
  # print(final_split)
  final_professions = [anchor_content.lower() for anchor_content in final_split]
  return final_professions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def check_wiki_professions(list_profession):
  required_professions = PROFESSION_TYPES

  check_individual_prof = []
  for x in required_professions:
    final_check= [(x in y) for y in list_profession]
    check_individual_prof.append(any(final_check))
    # print(final_check)
    # print(x)
  
  if (any(check_individual_prof)==True):
    # print(check_individual_prof)
    return True
  else:
    return False

In [ ]:
"""  From top 5 anchor we get most probable anchor in that show
     Make sure that the input anchors list follows given format. Here's an example
     Anchor_list = "Colleen-Williams_Ashwini-Bhave_Elaine-Quijano_Becky-Hobbs_Heidi-Collins"
"""
def get_correct_anchor(top_5):
  top_5_list = (top_5.replace('-',' ').replace('_',',')).split(',')
  # print(top_5_list)
  top_5_list = list(filter(None, top_5_list))
  for x in top_5_list:
    # print(x)
    # if Anchor_Name in Store_name:
    # pqe = check_wiki_professions(get_wiki_professions(x)) is True:

    if check_wiki_professions(get_wiki_professions(x)) is True:
      Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
      Store_name = Read_Namebasics.primaryName.tolist()
      if x in Store_name:
        return x 

    # if pqe is True:
      # list_of_selected_anchor.append(x)
     
  return '' 

In [ ]:
## get finals anchors from the multiple probable hosts

def Final_Anchor(txt_path):
  filter_probable_list = []
  all_lines = []                              
  with open (txt_path) as myfile:  
      for line in myfile:                   
          all_lines.append(line)
  # print(len(all_lines))                      
  last_line = all_lines[len(all_lines)-1].split('INF|')[1].split('\n')[0].replace(' ','')
  # print(last_line)
  avoid_num = ''.join([i for i in last_line if not i.isdigit()])
  probable_host_list = avoid_num.split('probable_host:')
  probable_host_list = list(filter(None, probable_host_list))

  # print(probable_host_list)

  for i in probable_host_list:
    filter_probable_list.append(get_correct_anchor(i))
    filter_probable_list = list(filter(None, filter_probable_list))
  return filter_probable_list

In [ ]:
#here we are getting all anchors present in a video
def get_list_anchors(folder_path):#folder path = where all txt files placed of a show 
  list_of_anchors = []
  list_txt_files = os.listdir(folder_path)
  for i in list_txt_files:
    txt_file_path = folder_path+'/'+i
    # print(Final_Anchor(txt_file_path))
    list_of_anchors.append(Final_Anchor(txt_file_path))
    print("Wait it will take 10-15 min")
  return list_of_anchors

[['Ellen DeGeneres'],
 ['Colleen Williams'],
 ['Randy Credico', 'Simcha Jacobovici'],
 ["Nancy O'Dell"],
 ['Colleen Williams', 'Craig Charles'],
 ['Matt Lauer', 'Katie Couric', 'Russell Baker'],
 ['Simcha Jacobovici']]

# **Get Networks**

In [ ]:
#this will give us network name either from anchor or show
def get_channel(name):
  try:
    anchor_content = ((((wiki.page(name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. ')))
  
  except wiki.exceptions.PageError as p:
    if PAGE_ERROR in str(p):
      return NONE
  
  except wiki.exceptions.DisambiguationError as e:
    list_disambiguation_error = e.options
    for anchor_name in list_disambiguation_error:
      if anchor_name.find(DISAMBIGUATION_ERROR)!= -1:
        return NONE
      
      else:
        anchor_content = ((((wiki.page(anchor_name,auto_suggest=False)).content).replace(' and ',', ').replace('.','. ')))
  # print(anchor_content)
  Networks_list = NETWORK_TYPES
  Network_filter = []
    
  for network in Networks_list:
    # print(word)
    if anchor_content.find(network)!=-1:
      Network_filter.append(network)
  return Network_filter

In [ ]:
def get_majority_network(anchors_list):
  list_of_network = []
  for anchorlist in anchors_list:
    for anchor in anchorlist:
      # print(a)
      list_of_network.append(get_channel(anchor))
  list_of_network = list(filter(None,list_of_network))
  print(list_of_network)

  X ={} #dict contains networks and respected votes 
  for network in list_of_network:
    for particular in network:
      if particular in X:
        X[particular] +=1
      else:
        X[particular] = 1
  print(X)   
  Max_votes = max(X.values())
  # print(Max_votes)
  Majority_Network = [particular for particular, votes in X.items() if votes == Max_votes]
  return Majority_Network

## **Get Shows**

In [ ]:
def get_nconst(Anchor_Name,Pull_Year):

  Read_Titlebasics = pd.read_table(TITLE_BASICS, sep='\t')
  Read_Namebasics = pd.read_table(NAME_BASICS, sep='\t')
  
  Read_Namebasics.primaryProfession = Read_Namebasics.primaryProfession.fillna("Null")
  filter_year = Read_Namebasics[Read_Namebasics.deathYear >= Pull_Year]
  Search_name = filter_year[filter_year.primaryName == Anchor_Name]
  # filter_profession = Search_name[Search_name.primaryProfession.str.contains(IMDB_PROFESSION_TYPES)]
  # return filter_profession
  # return Search_name
  # return len(filter_profession.index)
  if len(Search_name.index) == 1:
    for index, row in Search_name.iterrows():
      prepare_list_kft = row.knownForTitles.split(',')
      return [row.nconst,prepare_list_kft]

  else:
     
    titleakas = pd.read_table(TITLE_AKAS, sep='\t')
    titleakas.region = titleakas.region.fillna(REGION_TYPE)
    filter_region = titleakas[titleakas.region == REGION_TYPE]
    filter_region_titleid = filter_region.titleId.tolist()

    for index, row in Search_name.iterrows():
      prepare_list_kft = row.knownForTitles.split(',')
      for i in prepare_list_kft:
        take_title = Read_Titlebasics[Read_Titlebasics.tconst == i]
        if i in filter_region_titleid:
          check_genres = take_title[take_title.genres.str.contains(GENRE_TYPES)]
          if len(check_genres.index) == 1:
            # print(prepare_list_kft)
            # print(check_genres)
            return [row.nconst,prepare_list_kft]
    return [NONE]


In [ ]:
def get_all_shows(nConst):
  chunksize = 6*10**6
  titleprincipal = pd.read_table(TITLE_PRINCIPALS, chunksize = chunksize)
  all_shows = pd.DataFrame(columns=['tconst', 'ordering', 'nconst', 'category', 'job', 'characters'])
  #Initializing with the same columns as title.principals
  for i,chunk in enumerate((titleprincipal)):
    req_shows = chunk[chunk['nconst'] == nConst]
    print(' {} shows in chunk {}'.format(len(req_shows), i))
    all_shows = all_shows.append(req_shows, ignore_index = True)
    # filter_category = all_shows[all_shows.category == 'self']

  return all_shows.tconst.tolist()

In [ ]:
# get final titles afters union or filter
def get_show_names(Anchor_Name,Pull_Year):
  store_US_titles = []

  store_nconst = get_nconst(Anchor_Name,Pull_Year)

  if store_nconst[0] == NONE:
    return None

  titlecrew = pd.read_table('/content/drive/My Drive/Datasets/title.crew.tsv/data.tsv' ,sep='\t')
  nconst_in_dir = titlecrew[titlecrew.directors.str.contains(store_nconst[0])]
  nconst_in_writer = titlecrew[titlecrew.writers.str.contains(store_nconst[0])]
  tconst_in_crew = list(set().union(nconst_in_dir.tconst.tolist(),nconst_in_writer.tconst.tolist()))

  # Combine_list_1 = list(set().union(store_nconst[1],get_all_shows(store_nconst[0])))
  Combine_list_2 = set().union(store_nconst[1],get_all_shows(store_nconst[0]),tconst_in_crew)

# Here applying US(region) filter in the shows

  titleakas = pd.read_table(TITLE_AKAS, sep='\t')
  titleakas.region = titleakas.region.fillna(REGION_TYPE)
  filter_region = titleakas[titleakas.region == REGION_TYPE]
  filter_region_titleid = filter_region.titleId.tolist()
  for x in Combine_list_2:
    if x in filter_region_titleid:
      store_US_titles.append(x)

  # return store_US_titles
  tConsts= set(store_US_titles)

  chunksize = 6*10**6

  titlebasics = pd.read_table(TITLE_BASICS, chunksize = chunksize)
  Show_Names = pd.DataFrame(columns=['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes', 'genres'])
  for i,chunk in enumerate(titlebasics):
    show_names = chunk[chunk['tconst'].isin(tConsts)]
    # show_names = chunk[chunk['tconst'] == 'tt2788528']
    print('{} shows in chunk {}'.format(len(show_names), i))
    Show_Names = Show_Names.append(show_names, ignore_index = True)
    # print(Show_Names)
    filter_title_type = Show_Names[Show_Names['titleType']==TITLE_TYPE]
    filter_genres = filter_title_type[filter_title_type.genres.str.contains(GENRE_TYPES)]
    filter_end_year = filter_genres[filter_genres.endYear >= Pull_Year]
    filter_start_year = filter_end_year[filter_end_year.startYear.apply(str) <=Pull_Year]
  return filter_start_year 

In [ ]:
def final_show(anchor_list):
  X = {} #dict contains show name and their votes
  for host in anchor_list:
    print(host)
    store_show_names = get_show_names(host,'2006')
    if store_show_names is None:
      title = SHOW_NOT_IN_IMDB
      if title in X:
        X[title] +=1
      else:
        X[title] = 1
    
    else:
      for title in store_show_names.primaryTitle.tolist():
        if title in X:
          X[title] +=1
        else:
          X[title] = 1
  if bool(X) is False:
    return NONE
  else:
    Max_votes = max(X.values())
    # print(Max_votes)
    Majority_Shows = [title for title, votes in X.items() if votes==Max_votes]
    if len(Majority_Shows) is 1:
      print(Majority_Shows)
      # return Majority_Shows
    else:
      Open_Subtitles = open("/content/drive/My Drive/Datasets/Subtitles/2006-01-04_0000_US_00000063_V2_MB7_VHS7_H11_MS.txt3")
      Read_Subtitles = Open_Subtitles.read()

      for show in Majority_Shows:
        if Read_Subtitles.find(show.upper())!=-1:
          print(show)
          return show
          

      for show in Majority_Shows:
        for network in get_majority_network(anchor_list):
          if show.find(network)!=-1:
            print(show)
            return show
      
      # for show in Majority_Shows:
      #   call_channel = get_channel(show)
      #   if call_channel:
      #     for network in call_channel:
      #       if network == ('NBC'):
      #         return show 

        # else:
        #   return "network is not available"


In [ ]:
final_show(list_of_anchors)

In [ ]:
list_of_anchors = [["Heidi Collins"]]
for i in list_of_anchors:
  final_show(i)